In [90]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch as t
import torch.nn as nn
import warnings
import math
import itertools
import copy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

# 1. Preproccessing data

In [91]:
# 1. Data Import and Selection
columns_required = [
    'age', 'c_charge_degree', 'race', 'age_cat', 'sex',
    'priors_count', 'is_recid', 'two_year_recid',
    'c_jail_in', 'c_jail_out'
]
dataset = pd.read_csv("compas-scores-two-years.csv", usecols=columns_required)
print("Initial dataset shape:", dataset.shape)

# Checking the unique values of race to ensure all necessary mappings are accounted for
print("Unique race values in dataset:", dataset['race'].unique())

# 2. Feature Encoding
def encode_features(df):
    race_mapping = {'African-American': 0, 'Caucasian': 1}
    sex_mapping = {'Male': 1, 'Female': 0}
    age_cat_mapping = {'Less than 25': 0, '25 - 45': 1, 'Greater than 45': 2}
    c_charge_degree_mapping = {'F': 0, 'M': 1}

    # Keep records for African-American and Caucasian
    df_filtered = df[df['race'].isin(race_mapping.keys())]
    print("Shape after filtering for race:", df_filtered.shape)

    df_filtered['race'] = df_filtered['race'].map(race_mapping)
    df_filtered['sex'] = df_filtered['sex'].map(sex_mapping)
    df_filtered['age_cat'] = df_filtered['age_cat'].map(age_cat_mapping)
    df_filtered['c_charge_degree'] = df_filtered['c_charge_degree'].map(c_charge_degree_mapping)

    return df_filtered

processed_data = encode_features(dataset)

# 3. Calculating Length of Stay
processed_data['c_jail_in'] = pd.to_datetime(processed_data['c_jail_in'])
processed_data['c_jail_out'] = pd.to_datetime(processed_data['c_jail_out'])
processed_data['length_of_stay'] = (processed_data['c_jail_out'] - processed_data['c_jail_in']).dt.days

# Apply the specified bins to the length of stay
processed_data['length_of_stay'] = processed_data['length_of_stay'].apply(
    lambda days: 0 if days <= 7 else (2 if days > 90 else 1)
)

# 5. Processing Prior Crime Counts
processed_data['priors_count'] = processed_data['priors_count'].apply(
    lambda count: 0 if count == 0 else (2 if count > 3 else 1)
)

# 6. Handling Duplicate Values
final_dataset = processed_data.drop_duplicates()
print("Final dataset shape after dropping duplicates:", final_dataset.shape)

# 7. # Split the dataset into training, validation, and test sets
train_features, remaining_features, train_target, remaining_target = train_test_split(
    final_dataset.drop(columns=["two_year_recid"]), final_dataset["two_year_recid"], train_size=5/7.0
)
validation_features, test_features, validation_target, test_target = train_test_split(
    remaining_features, remaining_target, test_size=0.5
)


Initial dataset shape: (7214, 10)
Unique race values in dataset: ['Other' 'African-American' 'Caucasian' 'Hispanic' 'Native American'
 'Asian']
Shape after filtering for race: (6150, 10)
Final dataset shape after dropping duplicates: (6117, 11)


# 2. Choose Baseline Model

In [92]:
# Creating extended dataset
extended_dataset = final_dataset

# Defining feature sets
core_features = ['c_charge_degree', 'age_cat', 'sex', 'race', 'length_of_stay']
extended_features_list = ["priors_count"] + core_features

# Splitting the extended dataset into training, validation, and test sets
train_set = extended_dataset[:int(len(extended_dataset) * (5/7))]
validation_set = extended_dataset[int(len(extended_dataset) * (6/7)):]
test_set = extended_dataset[int(len(extended_dataset) * (5/7)):int(len(extended_dataset) * (6/7))]

# Preparing training, testing, and validation data
train_data_core = train_set[core_features]
train_data_extended = train_set[extended_features_list]
train_labels = train_set["two_year_recid"].to_numpy()
train_priors_count = train_set["priors_count"]

test_data_core = test_set[core_features]
test_data_extended = test_set[extended_features_list]
test_labels = test_set["two_year_recid"].to_numpy()
test_priors_count = test_set["priors_count"]

validation_data_core = validation_set[core_features]
validation_data_extended = validation_set[extended_features_list]
validation_labels = validation_set["two_year_recid"].to_numpy()
validation_priors_count = validation_set["priors_count"]

# Displaying the first few rows of validation data and its shape
print(validation_data_core.head())
print("Validation data shape:", np.shape(validation_data_core))

# Logistic Regression models
clf_priors_count = LogisticRegression().fit(train_data_extended, train_labels)
accuracy_priors_count = clf_priors_count.score(validation_data_extended, validation_labels)

clf_without_priors_count = LogisticRegression().fit(train_data_core, train_labels)
accuracy_without_priors_count = clf_without_priors_count.score(validation_data_core, validation_labels)
accuracy_priors_count, accuracy_without_priors_count

      c_charge_degree  age_cat  sex  race  length_of_stay
6184                1        1    0     0               0
6186                0        0    1     1               1
6187                0        1    1     0               0
6189                0        2    1     0               1
6190                0        1    1     0               0
Validation data shape: (874, 5)


(0.6556064073226545, 0.5983981693363845)

Clearly, the model with priors count feature has higher accuracy, which means the priors count is a senstive feature, and we need to include it.

# 3. Define functions

In [93]:
# Clear duplicates
final_dataset = processed_data.drop_duplicates()
print("Final dataset shape after dropping duplicates:", final_dataset.shape)

def uni_values_array(arr):
    return [np.unique(arr[:, col]).tolist() for col in range(arr.shape[1])]

def power_func(seq):
    result = [[]]
    for elem in seq:
        result.extend([x + [elem] for x in result])
    return result

def unique_information(array_1, array_2):
    assert array_1.shape[0] == array_2.shape[0], "Arrays must have the same number of rows"

    concated_array = np.concatenate((array_1, array_2), axis=1)
    cartesian_product = itertools.product(*uni_values_array(concated_array))
    IQ = 0
    for i in cartesian_product:
        mask = (concated_array == i).all(axis=1)
        r1_r2 = np.mean(mask)
        r1 = np.mean((array_1 == i[:array_1.shape[1]]).all(axis=1))
        r2 = np.mean((array_2 == i[array_1.shape[1]:]).all(axis=1))
        IQ_iter = r1_r2 * np.log(r1_r2 / r1) / r1 if r1_r2 > 0 and r1 > 0 and r2 > 0 else 0
        IQ += np.abs(IQ_iter)
    return IQ

def unique_infor_condi(array_1, array_2, conditional):
    assert (array_1.shape[0] == array_2.shape[0]) and (array_1.shape[0] == conditional.shape[0]), "Arrays must have the same number of rows"

    concated_array_all = np.concatenate((array_1, array_2, conditional), axis=1)
    cartesian_product = itertools.product(*uni_values_array(concated_array_all))
    IQ = 0
    for i in cartesian_product:
        mask_all = (concated_array_all == i).all(axis=1)
        mask_1 = (array_1 == i[:array_1.shape[1]]).all(axis=1)
        mask_2_cond = (concated_array_all[:, array_1.shape[1]:] == i[array_1.shape[1]:]).all(axis=1)
        r1_r2 = np.mean(mask_all)
        r1 = np.mean(mask_1)
        r2 = np.mean(mask_2_cond)
        r1_given_r2 = np.mean(mask_1[mask_2_cond]) if np.any(mask_2_cond) else 0
        IQ_iter = r1_r2 * np.log(r1_r2 / r2) / r1_given_r2 if r1_r2 > 0 and r1 > 0 and r2 > 0 and r1_given_r2 > 0 else 0
        IQ += np.abs(IQ_iter)
    return IQ

def accuracy_coef(y, x_s, x_s_c, A):
    conditional = np.concatenate((x_s_c, A), axis=1)
    return unique_infor_condi(y, x_s, conditional)

def discrimination_coef(y, x_s, A):
    x_s_a = np.concatenate((x_s, A), axis=1)
    return unique_information(y, x_s_a) * unique_information(x_s, A) * unique_infor_condi(x_s, A, y)

def marginal_accuracy_coef(y_train, x_train, A, set_tracker):
    n_features = x_train.shape[1]
    shapley_value = 0
    for sc_idx in itertools.chain.from_iterable(itertools.combinations(range(n_features), r) for r in range(n_features)):
        if set_tracker in sc_idx:
            continue
        coef = math.factorial(len(sc_idx)) * math.factorial(n_features - len(sc_idx) - 1) / math.factorial(n_features)
        sc_idx_with_i = list(sc_idx) + [set_tracker]
        vTU = accuracy_coef(y_train.reshape(-1, 1), x_train[:, sc_idx_with_i], x_train[:, list(set(range(n_features)) - set(sc_idx_with_i))], A.reshape(-1, 1))
        vT = accuracy_coef(y_train.reshape(-1, 1), x_train[:, sc_idx], x_train[:, list(set(range(n_features)) - set(sc_idx))], A.reshape(-1, 1))
        shapley_value += coef * (vTU - vT)
    return shapley_value

def marginal_discrimination_coef(y_train, x_train, A, set_tracker):
    n_features = x_train.shape[1]
    shapley_value = 0
    for sc_idx in itertools.chain.from_iterable(itertools.combinations(range(n_features), r) for r in range(n_features)):
        if set_tracker in sc_idx:
            continue
        coef = math.factorial(len(sc_idx)) * math.factorial(n_features - len(sc_idx) - 1) / math.factorial(n_features)
        sc_idx_with_i = list(sc_idx) + [set_tracker]
        vTU = discrimination_coef(y_train.reshape(-1, 1), x_train[:, sc_idx_with_i], A.reshape(-1, 1))
        vT = discrimination_coef(y_train.reshape(-1, 1), x_train[:, sc_idx], A.reshape(-1, 1))
        shapley_value += coef * (vTU - vT)
    return shapley_value

Final dataset shape after dropping duplicates: (6117, 11)


# 4. Calculate Shapley Value

In [94]:
# Calculate Shapley values for each feature
shapley_acc = []
shapley_disc = []
features = extended_features_list  # Using the extended set of features
for i in range(len(features)):
    acc_i = marginal_accuracy_coef(train_labels, train_data_extended[features].to_numpy(), train_data_extended['race'].to_numpy(), i)
    disc_i = marginal_discrimination_coef(train_labels, train_data_extended[features].to_numpy(), train_data_extended['race'].to_numpy(), i)
    shapley_acc.append(acc_i)
    shapley_disc.append(disc_i)

# Create a DataFrame to display the Shapley values for each feature
shapley = pd.DataFrame(list(zip(features, shapley_acc, shapley_disc)),
                       columns=["Feature", "Accuracy", 'Discrimination'])

# Calculate and display the fairness utility score for different alpha values
alpha_values = [0.000001, 0.00001, 0.0001]
for alpha in alpha_values:
    alpha_df = pd.DataFrame(list(zip(features, shapley_acc, shapley_disc, fairness_utility_score(shapley['Accuracy'], shapley['Discrimination'], alpha))),
                            columns=["Feature", "Accuracy", 'Discrimination', 'F_score'])
    print(f"Alpha = {alpha}")
    print(alpha_df)

Alpha = 1e-06
           Feature      Accuracy  Discrimination   F_score
0     priors_count  0.000000e+00     8090.544032 -0.008091
1  c_charge_degree -9.992007e-17     6308.746881 -0.006309
2          age_cat -1.147230e-16     7990.621865 -0.007991
3              sex  1.295260e-16     6057.511767 -0.006058
4             race -2.035409e-16   -36275.753860  0.036276
5   length_of_stay -1.406282e-16     7828.329315 -0.007828
Alpha = 1e-05
           Feature      Accuracy  Discrimination   F_score
0     priors_count  0.000000e+00     8090.544032 -0.080905
1  c_charge_degree -9.992007e-17     6308.746881 -0.063087
2          age_cat -1.147230e-16     7990.621865 -0.079906
3              sex  1.295260e-16     6057.511767 -0.060575
4             race -2.035409e-16   -36275.753860  0.362758
5   length_of_stay -1.406282e-16     7828.329315 -0.078283
Alpha = 0.0001
           Feature      Accuracy  Discrimination   F_score
0     priors_count  0.000000e+00     8090.544032 -0.809054
1  c_charge_d

In [95]:
# 1. Remove specific features and train a logistic regression model.
train_data_vc = train_data_extended.drop(["race"], axis=1)
validation_data_vc = validation_data_extended.drop(["race"], axis=1)
clf_vc = LogisticRegression(random_state=0).fit(train_data_vc, train_labels)
accuracy_vc = clf_vc.score(validation_data_vc, validation_labels)

train_data_ls = train_data_extended.drop(["length_of_stay"], axis=1)
validation_data_ls = validation_data_extended.drop(["length_of_stay"], axis=1)
clf_ls = LogisticRegression(random_state=0).fit(train_data_ls, train_labels)
accuracy_ls = clf_ls.score(validation_data_ls, validation_labels)

# 2. Define a calibration metric function.
def MyCalibration(sensitive_attr, y_pred, y_true):
    cau_index = np.where(sensitive_attr == 1)[0]
    african_index = np.where(sensitive_attr == 0)[0]

    y_pred_cau = y_pred[cau_index]
    y_true_cau = y_true[cau_index]
    Acc_cau = sum(y_pred_cau == y_true_cau)/len(y_pred_cau)

    y_pred_african = y_pred[african_index]
    y_true_african = y_true[african_index]
    Acc_african = sum(y_pred_african == y_true_african)/len(y_pred_african)

    calibration = abs(Acc_cau - Acc_african)
    return(calibration)

# 3. Evaluate the impact of removing each feature on model performance.
Accuracy_lr = []
Calibration_lr = []
for feature in ['base'] + extended_features_list:
    if feature == 'base':
        clf = LogisticRegression(random_state=0).fit(train_data_extended, train_labels)
    else:
        train_data_subset = train_data_extended.drop([feature], axis=1)
        clf = LogisticRegression(random_state=0).fit(train_data_subset, train_labels)

    test_data_subset = test_data_extended.drop([feature], axis=1) if feature != 'base' else test_data_extended
    Accuracy_lr.append(clf.score(test_data_subset, test_labels))
    Calibration_lr.append(MyCalibration(test_data_extended['race'], clf.predict(test_data_subset), test_labels))

# 4. Create a conclusions DataFrame.
Conclusion_lr = pd.DataFrame(list(zip(['base'] + extended_features_list, Accuracy_lr, Calibration_lr)),
                             columns=["Eliminating Feature", "Accuracy", "Calibration"])

display(Conclusion_lr)

,Eliminating Feature,Accuracy,Calibration
0,base,0.653318,0.037871
1,priors_count,0.592677,0.043629
2,c_charge_degree,0.654462,0.035161
3,age_cat,0.624714,0.006944
4,sex,0.652174,0.045271
5,race,0.663616,0.032241
6,length_of_stay,0.668192,0.044852


## Simplified Analysis of Feature Combinations in Predictive Model

### Combination 1: `priors_count` and `length_of_stay`
- **`priors_count`**: Removing this feature significantly decreases accuracy, indicating its importance. Yet, its removal might reduce overfitting or bias.
- **`length_of_stay`**: Its removal increases accuracy but also calibration disparity, hinting at its role in introducing subgroup inconsistencies.

### Combination 2: `length_of_stay` and `sex`
- **`length_of_stay`**: Similar to Combination 1, its removal slightly improves accuracy but raises calibration disparity, suggesting bias introduction.
- **`sex`**: Its removal barely affects accuracy but significantly increases calibration disparity, indicating minimal contribution to model bias.

### Combination 3: `age_cat` and `length_of_stay`
- **`age_cat`**: Lowest calibration disparity. Removing it increases model fairness but decreases accuracy.
- **`length_of_stay`**: Consistently introduces bias with lesser impact on accuracy.

### Combination 4: `length_of_stay` and `race`
- **`length_of_stay`**: As previously noted, introduces bias.
- **`race`**: Its removal improves accuracy but also raises calibration disparity. Consider removal for reducing racial influence in sensitive applications like recidivism prediction.


In [96]:
# Final model training and evaluation/trial.
train_data_final = train_data_extended.drop(["length_of_stay", "sex"], axis=1)
validation_data_final = validation_data_extended.drop(["length_of_stay", "sex"], axis=1)
clf_final = LogisticRegression(random_state=0).fit(train_data_final, train_labels)
accuracy_final = clf_final.score(validation_data_final, validation_labels)
display(accuracy_final)

0.6590389016018307

In [97]:
train_data_final = train_data_extended.drop(["length_of_stay", "priors_count"], axis=1)
validation_data_final = validation_data_extended.drop(["length_of_stay", "priors_count"], axis=1)
clf_final = LogisticRegression(random_state=0).fit(train_data_final, train_labels)
accuracy_final = clf_final.score(validation_data_final, validation_labels)
display(accuracy_final)

0.5938215102974829

In [98]:
train_data_final = train_data_extended.drop(["length_of_stay", "age_cat"], axis=1)
validation_data_final = validation_data_extended.drop(["length_of_stay", "age_cat"], axis=1)
clf_final = LogisticRegression(random_state=0).fit(train_data_final, train_labels)
accuracy_final = clf_final.score(validation_data_final, validation_labels)
display(accuracy_final)

0.6521739130434783

In [99]:
train_data_final = train_data_extended.drop(["length_of_stay", "race"], axis=1)
validation_data_final = validation_data_extended.drop(["length_of_stay", "race"], axis=1)
clf_final = LogisticRegression(random_state=0).fit(train_data_final, train_labels)
accuracy_final = clf_final.score(validation_data_final, validation_labels)
display(accuracy_final)

0.665903890160183

In [100]:
train_data_final = train_data_extended.drop(["length_of_stay"], axis=1)
validation_data_final = validation_data_extended.drop(["length_of_stay"], axis=1)
clf_final = LogisticRegression(random_state=0).fit(train_data_final, train_labels)
accuracy_final = clf_final.score(validation_data_final, validation_labels)
display(accuracy_final)

0.6590389016018307

### Conclusion
After testing four combinations and analyzing the recurring feature `length_of_stay`, the best outcome is achieved by simultaneously removing `length_of_stay` and `race`. This approach optimizes model performance, particularly in contexts where reducing racial bias is crucial.